In [1]:
import pickle
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import numpy as np
import matplotlib.mlab as mlab

dd = pickle.load(open("dlmr.pickle", 'rb'))
dl, mr = dd

In [44]:
# the random data
mr = np.array(mr)
dl = np.array(dl)

mr_major = 3
dl_major = dl[mr < mr_major]
dl_minor = dl[mr > mr_major]

yll, ylr = (-0.4, 0.4)

same_x = True
# same x scale 
if same_x:
    hist_major, bin_edges = np.histogram(dl_major, bins=10, range=(yll,ylr))
    hist_minor, bin_edges = np.histogram(dl_minor, bins=10, range=(yll,ylr))

    mM = max(hist_major)
    mm = max(hist_minor)

    width_M = mM/(mM + mm) * 0.4
    width_m = mm/(mM + mm) * 0.4
else:
    width_M = 0.2
    width_m = 0.2

# definitions for the axes
left, width = 0.1, 0.4
bottom, height = 0.1, 0.7

rect_scatter = [left, bottom, width, height]
rect_major = [left + width, bottom, width_M, height]
rect_minor = [left + width + width_M, bottom, width_m, height]

# start with a rectangular Figure
plt.figure(1, figsize=(16,8))

axScatter = plt.axes(rect_scatter)
axHist1 = plt.axes(rect_major)
axHist2 = plt.axes(rect_minor)

# no labels
nullfmt   = NullFormatter() # no labels
axHist1.yaxis.set_major_formatter(nullfmt)
axHist2.yaxis.set_major_formatter(nullfmt)


# the scatter plot:
axScatter.scatter(mr[mr < mr_major], dl[mr < mr_major], color='blue')
axScatter.scatter(mr[mr > mr_major], dl[mr > mr_major], color='green')
axScatter.axvline(mr_major, c='grey', linestyle='dashed', linewidth=2) # Major / Minor
axScatter.set_xlabel("Merger mass ratio", fontsize=14)
axScatter.set_ylabel(r"$\Delta \lambda_{R_{e}}$", fontsize=16)

axScatter.set_xlim((0.5, 10.5)) # above 1
axScatter.set_ylim((yll, ylr))

(-0.4, 0.4)

In [45]:
#axHist1.barh(np.arange(len(hist)), hist_major)
#axHist1.hist()

hM = axHist1.hist(dl_major, orientation='horizontal', color='blue', range=(yll,ylr))
axHist1.set_xlim((1.1*max(hM[0]), 0))

def gaussian_fit(ax, data, dx, color='green'):
    mean = np.mean(data)
    variance = np.var(data)
    sigma = np.sqrt(variance)
    x = np.linspace(min(data), max(data), 100)
    scale = len(data)*dx
    ax.plot(mlab.normpdf(x,mean,sigma)*scale, x, color=color)
    ax.text(0.1, 0.9, "mean = {:.2f}\n sig = {:.2f}".format(mean,sigma),
            horizontalalignment='left',
            verticalalignment='top',
            transform=ax.transAxes)

def fwhm(xx, curve):
    from scipy.interpolate import UnivariateSpline
    x = np.linspace(min(xx), max(xx), 100)
    spline = UnivariateSpline(x, curve-np.max(curve)/2, s=0) # Find FWHM location
    r1, r2 = spline.roots()
    
gaussian_fit(axHist1, dl_major, hM[1][1] - hM[1][0], color='cyan')


#axHist2.barh(np.arange(len(hist_minor)), hist_minor, color='green')
hm = axHist2.hist(dl_minor, orientation='horizontal', color='green', range=(yll,ylr))
axHist2.set_xlim((0, 1.1*max(hm[0])))
gaussian_fit(axHist2, dl_minor, hm[1][1] - hm[1][0], color='greenyellow')


axHist1.set_ylim(axScatter.get_ylim())
axHist2.set_ylim(axScatter.get_ylim())
axHist2.set_xlabel("# of mergers", x=0)
#fig.tight_layout()
#plt.show()
plt.savefig("delta_lambda.pdf", bbox_inches='tight') # eps does NOT support transparency!
plt.close()

/home/hoseung/.local/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
